In [1]:
!pip install crewai requests flask pandas matplotlib
!pip install load_dotenv
!pip install qwen
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.1 MB/s eta 0:

In [ ]:
#set up API credentials
TRELLO_API_KEY=your_trello_api_key
TRELLO_TOKEN=your_trello_token
SLACK_BOT_TOKEN=your_slack_bot_token
SLACK_CHANNEL_ID=your_channel_id

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
TRELLO_API_KEY = os.getenv("TRELLO_API_KEY")
TRELLO_TOKEN = os.getenv("TRELLO_TOKEN")
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
SLACK_CHANNEL_ID = os.getenv("SLACK_CHANNEL_ID")

In [ ]:
#authenticating with Trello API
import requests

def get_trello_boards():
    url = "https://api.trello.com/1/members/me/boards"
    params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}
    response = requests.get(url, params=params)
    return response.json()

def get_trello_lists(board_id):
    url = f"https://api.trello.com/1/boards/{board_id}/lists"
    params = {"key": TRELLO_API_KEY, "token": TRELLO_TOKEN}
    response = requests.get(url, params=params)
    return response.json()

In [ ]:
#test these functions to ensure they work
boards = get_trello_boards()
print("Your Trello Boards:", boards)

In [ ]:
#creating the input agent with Qwen
from qwen_api import QwenClient

client = QwenClient(api_key="your_qwen_api_key")

def parse_user_input(user_message):
    response = client.send_message(user_message)
    intent = response['intent']  # e.g., "move_task", "create_task"
    details = response['details']  # e.g., {"task_id": 987, "status": "Done"}
    return intent, details

In [ ]:
user_message = "Move Task 987 to Done"
intent, details = parse_user_input(user_message)
print(intent, details)  # Output: move_task {'task_id': 123, 'status': 'Done'}

In [ ]:
#automating tasks with the manager agent
def update_trello_card(card_id, list_id):
    url = f"https://api.trello.com/1/cards/{card_id}"
    params = {
        "key": TRELLO_API_KEY,
        "token": TRELLO_TOKEN,
        "idList": list_id
    }
    response = requests.put(url, params=params)
    return response.status_code == 200

In [ ]:
#map list names to IDs
list_mapping = {list['name']: list['id'] for list in get_trello_lists(board_id)}

In [ ]:
update_trello_card(card_id="123", list_id=list_mapping["Done"])

In [ ]:
#generating daily standup summaries
def generate_daily_summary(cards):
    summary = "Daily Stand-Up Summary:\n"
    for card in cards:
        summary += f"- {card['name']}: {card['status']}\n"
    return summary

def send_to_slack(message):
    url = "https://slack.com/api/chat.postMessage"
    headers = {"Authorization": f"Bearer {SLACK_BOT_TOKEN}"}
    payload = {"channel": SLACK_CHANNEL_ID, "text": message}
    requests.post(url, headers=headers, json=payload)

In [ ]:
#fetch cards and send summary
cards = get_trello_cards(board_id)
summary = generate_daily_summary(cards)
send_to_slack(summary)

In [ ]:
#orchestrsing agents with crewai
from crewai import Agent, Workflow

input_agent = Agent(name="InputAgent", function=parse_user_input)
task_manager = Agent(name="TaskManager", function=update_trello_card)
reporting_agent = Agent(name="ReportingAgent", function=generate_daily_summary)

workflow = Workflow()
workflow.add_step(input_agent)
workflow.add_step(task_manager)
workflow.add_step(reporting_agent)

# Run the workflow
workflow.run(user_message="Move Task 987 to Done")